In [2]:
import pandas as pd
import random
import datetime

# 忽略pandas的性能警告
import warnings
from pandas.errors import PerformanceWarning
warnings.filterwarnings("ignore", category=PerformanceWarning)

In [3]:
# 读取数据,注意学号必须转为字符型，如果表顶部有多余的行可以设置skiprows跳过
df = pd.read_excel("./20xx年重修名单.xlsx",skiprows=0)
df["学号"]=df["学号"].astype(str)
df

,学号,课程名称
0,10000310,课程26
1,10000471,课程26
2,10000194,课程26
3,10000375,课程26
4,10000144,课程26
...,...,...
769,10000238,课程18
770,10000235,课程18
771,10000229,课程18
772,10000224,课程18


In [3]:
# 课程人数分析
df["课程名称"].value_counts().head(5)

课程1    111
课程2     87
课程3     86
课程4     63
课程5     62
Name: 课程名称, dtype: int64

In [4]:
# 学生课程数分析
df["学号"].value_counts().head(5)

10000000    7
10000001    6
10000002    6
10000003    6
10000004    5
Name: 学号, dtype: int64

In [5]:
# 构建【学号-课程】矩阵，参与考试标记为1，其他用0填充
test = pd.DataFrame()
for i,item in df.iterrows():
    test.loc[item["学号"], item["课程名称"]]=1
test = test.fillna(0)
# 按考试人数对课程排序
test = test[list(df["课程名称"].value_counts().index)]
test.head(5)

,课程1,课程2,课程3,课程4,课程5,课程6,课程7,课程8,课程9,课程10,...,课程54,课程55,课程56,课程57,课程58,课程59,课程60,课程61,课程62,课程63
10000310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000144,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# 求解
for j in range(10000):  # 求解1万次（实际上大约几十次就能找到满足要求的方案，或者一直找不到）
    test2 = test.copy()  # 复制矩阵表格
    for i in range(len(test2.columns)):
        columns = list(test2.columns[i+1:])  # 取出第i列以后的课程（前i列是已经无法合并的）
        if len(columns)==0:
            # 能合并的都已经合并了，提前退出
            break
        random.shuffle(columns)  # 打乱顺序，增加随机性
        for column2 in columns:  # 遍历可能合并的每一列
            column1 = test2.columns[i]  # 每次都要更新列名，因为列名可能会因为合并而变化

            # 这里给出了3个条件：1.被合并的两列不能有冲突；2.合并后每场的考试人数不能大于100人；3.合并后每场考试科目不能大于15门
            # 第1个条件必须满足才能合并，后2个条件可以根据需求修改
            if sum((test2[column1]>0) & (test2[column2]>0)) == 0 and (sum(test2[column1])+sum(test2[column2]))<=150 and (column1+"|"+column2).count("|")<10:
                # 将合并后的列插入到第i列
                test2.insert(i,column1+"|"+column2,test2[column1]+test2[column2])
                # 将被合并的列删除
                test2 = test2.drop(columns=[column1,column2])

    print(j,len(test2.columns),sep=",",end="\t") # 输出当前考试安排方案的场数

    # 退出循环的条件：合并后小于8列（即：最多安排8场考试）
    if len(test2.columns)<=8:
        # 保存到文件
        test2.to_excel("output"+str(len(test2.columns))+"-"+datetime.datetime.now().strftime("%Y%m%d%H%M%S")+".xlsx")
        break
test2.head(5)

0,9	1,10	2,8	

,课程1|课程32|课程28|课程60|课程33|课程10|课程23,课程2|课程42|课程43|课程50|课程16|课程29|课程57|课程59|课程38|课程48,课程3|课程39|课程20|课程62|课程49|课程54|课程58|课程51|课程55|课程63,课程4|课程53|课程56|课程24|课程35|课程41|课程52|课程45|课程47|课程36,课程5|课程46|课程15|课程31|课程18|课程22|课程37|课程44|课程30|课程27,课程6|课程14|课程19|课程26|课程40|课程17|课程34|课程61,课程7|课程11|课程25|课程21|课程12,课程8|课程13|课程9
10000310,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10000471,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10000194,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
10000375,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10000144,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
# 查看每场考试的科目数量和参考人数
for i,column in enumerate(test2.columns):
    print("第%d场考%d门，%d人参考"%(i+1,column.count("|")+1,sum(test2[column])))

第1场考7门，150人参考
第2场考10门，109人参考
第3场考10门，102人参考
第4场考10门，78人参考
第5场考10门，100人参考
第6场考8门，79人参考
第7场考5门，86人参考
第8场考3门，70人参考
